In [24]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
# IMPORT LIBRARIES
from io import BytesIO
import csv
import io
# PANDA AND BOTO3 MUST BE INCLUIDED IN THE REQUIREMENTS TEXT FILE
import pandas as pd
import boto3
import numpy as np
from decimal import Decimal
import json
import sys, os, base64, datetime, hashlib, hmac 
import sys, os, base64, datetime, hashlib, hmac 

#  AWS RESOURCES
try:
    from urlparse import urlparse, parse_qs
except ImportError:
    from urllib.parse import urlparse, parse_qs
    
client = boto3.client('sagemaker-runtime')
session = boto3.session.Session(region_name='us-east-1')
s3client = session.client('s3')

#GET API - CHALICE
#def handle_data():
# READ EMPLOYEE DATA BASE FROM S3 BUCKTE
resedb = s3client.get_object(Bucket='sagemaker-hremployeeattrition-0001', Key='database/EmployeeDatabase.csv')
dataset = pd.read_csv(io.BytesIO(resedb['Body'].read()), encoding='utf8')

#DATA SET PREPROCESSING (DROP COLUMNS NOT NEEDED FOR THE MODEL)
dataset.drop(['BusinessTravel', 'Attrition','EmployeeNumber','Atrition_Prediction_Value', 'Risk', 'EmployeeCount','StandardHours','Over18','JobRole'],axis = 1, inplace=True)
#TRANSFORM CATEGORICAL COLUMNS INTO DUMMY VARIABLES FOR THE MODEL TO WORK PROPERLY
cols_to_transform = ['Department', 'EducationField', 'Gender' , 'MaritalStatus' , 'OverTime']
dataset = pd.get_dummies(dataset, columns = cols_to_transform )
# RUN TRAINED MODEL FROM RUNNING SAGEMAKER ENDPOINT
input_file = StringIO()
dataset.to_csv(input_file, header=False, index=False)
res = client.invoke_endpoint(
                EndpointName='xgboost-2018-09-30-20-27-49-609',
                Body=input_file.getvalue(),
                ContentType='text/csv',
                Accept='Accept'
            )
# TRANSFORM PREDICTIONS INTO AN ARRAY, DATA FRAME
predictions_array = np.fromstring(res ['Body'].read(), sep=',')
dtattrition = pd.DataFrame(predictions_array)
dtattrition.columns = ['Atrition_Prediction_Value']
dtattrition = pd.DataFrame(dtattrition) 
# DEFINE THE CUTOFF AND THE INTERPRETAION FOR THE RESULT OF THE MODEL
def f(row):
    if Decimal(row[0]) > 0.21:
        val = "High"
    else:
        val = "Low"
    return val
dtattrition['Risk']= dtattrition.apply(f, axis=1) 
# APPLY SOME FORMATTING TO THE PREDICTIONS OUTPUT
dtattrition['Atrition_Prediction_Value'] = (dtattrition['Atrition_Prediction_Value'].round(4))*100
dtattrition['Atrition_Prediction_Value'] = dtattrition['Atrition_Prediction_Value'].astype(str)  + "%"
dtattrition = pd.DataFrame(dtattrition)
# READ ORINGINAL DATABASE AND LEFT JOIN WITH THE NEW PREDICTIONS OUTPUT
resedb = s3client.get_object(Bucket='sagemaker-hremployeeattrition-0001', Key='database/EmployeeDatabase.csv')
dataset = pd.read_csv(io.BytesIO(resedb['Body'].read()), encoding='utf8')
# DROP OLD COLUMNS RESULT AND REPLACE IT WITH THE NEW PREDICTIONS
dataset.drop(['Atrition_Prediction_Value', 'Risk'],axis = 1, inplace=True)
dataset = pd.concat([dtattrition,dataset], axis=1)
input_file2 = StringIO()
dataset.to_csv(input_file2,header=True, index=False)
# UPLOAD RESULTING MODEL IN AN S3 BUCKET REPLACING THE OLD EMPLOYEE DATABASE FILE 
#  ress3 = s3client.put_object(
      #      ACL='public-read-write',
       #     Bucket='sagemaker-hremployeeattrition-0001',
        #    Key='database/EmployeeDatabase.csv',
         #   Body=input_file2.getvalue(),
          #  ContentType='text/csv'
        #)

#   return res3

In [25]:
dataset

,Atrition_Prediction_Value,Risk,EmployeeNumber,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,49.45%,High,1,41,Yes,Travel_Rarely,1102,Sales,1,2,...,1,80,0,8,0,1,6,4,0,5
1,1.21%,Low,2,49,No,Travel_Frequently,279,Research & Development,8,1,...,4,80,1,10,3,3,10,7,1,7
2,75.79%,High,4,37,Yes,Travel_Rarely,1373,Research & Development,2,2,...,2,80,0,7,3,3,0,0,0,0
3,26.640000000000004%,High,5,33,No,Travel_Frequently,1392,Research & Development,3,4,...,3,80,0,8,3,3,8,7,3,0
4,6.87%,Low,7,27,No,Travel_Rarely,591,Research & Development,2,1,...,4,80,1,6,3,3,2,2,2,2
5,2.44%,Low,8,32,No,Travel_Frequently,1005,Research & Development,2,2,...,3,80,0,8,2,2,7,7,3,6
6,14.11%,Low,10,59,No,Travel_Rarely,1324,Research & Development,3,3,...,1,80,3,12,3,2,1,0,0,0
7,18.22%,Low,11,30,No,Travel_Rarely,1358,Research & Development,24,1,...,2,80,1,1,2,3,1,0,0,0
8,5.2299999999999995%,Low,12,38,No,Travel_Frequently,216,Research & Development,23,3,...,2,80,0,10,2,3,9,7,1,8
9,4.36%,Low,13,36,No,Travel_Rarely,1299,Research & Development,27,3,...,2,80,2,17,3,2,7,7,7,7
